In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LSTM, concatenate
from tensorflow.keras.optimizers import Adam
import logging
import re

# Suppress TensorFlow logging
logging.getLogger('tensorflow').setLevel(logging.ERROR)

def create_sequences(ts_data, n_past, n_future):
    """
    Creates sequences for time-series forecasting.
    ts_data: Numpy array of time-series features
    n_past: Number of past months to use as input
    n_future: Number of future months to predict
    """
    X_ts, y_ts = [], []
    for i in range(n_past, len(ts_data.columns) - n_future + 1):
        X_ts.append(ts_data.iloc[:, i - n_past:i].values)
        y_ts.append(ts_data.iloc[:, i:i + n_future].values)
    
    # This is a simplified example; for a real model, we'd need
    # to handle the sequence creation per-row and align it.
    # For this PoC, we will reshape the data directly.
    pass # See main() for the actual data prep logic

def build_forecasting_model(n_past_steps, n_features_ts, n_static_features, n_future_steps):
    """
    Defines the Keras Multi-Input Functional Model.
    """
    # --- Time-Series Input Path (LSTM) ---
    # This branch handles the historical price/rent sequences
    ts_input = Input(shape=(n_past_steps, n_features_ts), name='ts_input')
    lstm_layer = LSTM(50, activation='relu', return_sequences=False)(ts_input)
    lstm_out = Dropout(0.2)(lstm_layer)
    ts_dense = Dense(20, activation='relu')(lstm_out)

    # --- Static Data Input Path (Dense) ---
    # This branch handles crime, population, school scores, etc.
    static_input = Input(shape=(n_static_features,), name='static_input')
    dense_layer = Dense(128, activation='relu')(static_input)
    dense_out = Dropout(0.2)(dense_layer)
    dense_layer = Dense(64, activation='relu')(dense_out)
    static_dense = Dense(20, activation='relu')(dense_layer)

    # --- Concatenate Paths ---
    # Combine the wisdom from both branches
    combined = concatenate([ts_dense, static_dense])
    
    # --- Final Layers ---
    final_dense = Dense(50, activation='relu')(combined)
    
    # --- Output Layer ---
    # We want N neurons to predict the next N steps (e.g., 12 months)
    output = Dense(n_future_steps, activation='linear', name='output')(final_dense)
    
    # --- Build and Compile ---
    model = Model(inputs=[ts_input, static_input], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae'])
    
    return model

def get_letter_grade(score):
    """Converts a 0-100 score to a letter grade."""
    if score >= 90: return 'A'
    if score >= 80: return 'B'
    if score >= 70: return 'C'
    if score >= 60: return 'D'
    return 'F'

def calculate_scores(df):
    """
    Engineers 'Safety' and 'School' scores (0-100) and letter grades.
    This function modifies the DataFrame in place.
    """
    print("Calculating Risk/Opportunity Scores...")
    
    # --- 1. Safety Score (Lower is better) ---
    # We use 'crime_rate_per_100000'
    # We'll use quantiles: top 20% (lowest crime) get an A, bottom 20% get an F
    df['Safety_Score'] = pd.qcut(df['crime_rate_per_100000'],
                                 q=[0, .2, .4, .6, .8, 1.],
                                 labels=[95, 85, 75, 65, 55]).astype(float)
    # A simplified quantile scoring: 95 = A (top 20% safest), 55 = F (bottom 20%)
    
    # --- 2. School Score (Higher is better) ---
    # We'll use 'CountySchoolScore_y' and 'Proficiency'
    # Normalize both from 0-1 using min-max (robust to outliers)
    prof_min, prof_max = df['Proficiency'].min(), df['Proficiency'].max()
    school_min, school_max = df['CountySchoolScore_y'].min(), df['CountySchoolScore_y'].max()

    df['Proficiency_norm'] = (df['Proficiency'] - prof_min) / (prof_max - prof_min)
    df['SchoolScore_norm'] = (df['CountySchoolScore_y'] - school_min) / (school_max - school_min)
    
    # Combine them (e.g., 50/50 weights) and scale to 100
    df['School_Score'] = (0.5 * df['Proficiency_norm'] + 0.5 * df['SchoolScore_norm']) * 100
    
    # --- 3. Overall Score ---
    # We can create a weighted average, e.g., 60% Safety, 40% Schools
    df['Overall_Score'] = (0.6 * df['Safety_Score']) + (0.4 * df['School_Score'])
    
    # --- 4. Add Letter Grades ---
    df['Safety_Grade'] = df['Safety_Score'].apply(get_letter_grade)
    df['School_Grade'] = df['School_Score'].apply(get_letter_grade)
    df['Overall_Grade'] = df['Overall_Score'].apply(get_letter_grade)
    
    # Drop intermediate columns
    df.drop(['Proficiency_norm', 'SchoolScore_norm'], axis=1, inplace=True)
    
    print("Scoring complete.")
    return df

def find_date_columns(all_columns):
    """
    Identifies date-based columns (e.g., '1/31/2000' or '2015-01-31_zhvi')
    """
    # Regex to find 'YYYY-MM-DD_zhvi' or 'MM/DD/YYYY'
    date_regex = re.compile(r'(\d{4}-\d{2}-\d{2}_zhvi|\d{1,2}/\d{1,2}/\d{4})')
    zhvi_cols = sorted([col for col in all_columns if date_regex.match(col) and '_zhvi' in col])
    zori_cols = sorted([col for col in all_columns if date_regex.match(col) and '_zori' in col])
    
    # Simpler regex if the above fails
    if not zhvi_cols:
         date_regex_simple = re.compile(r'^\d{1,2}/\d{1,2}/\d{4}$')
         zhvi_cols = sorted([col for col in all_columns if date_regex_simple.match(col)])
         
    # Assuming zori columns are not present if zhvi fails to be found
    # This is a simplification; a real implementation would need robust column mapping
    
    # For this demo, we'll assume the simple M/D/YYYY format was found
    if not zori_cols and zhvi_cols:
        # Create dummy zori cols for the sake of model architecture
        # In a real case, you'd find the real '..._zori' cols
        zori_cols = zhvi_cols 
        
    return zhvi_cols, zori_cols


def main():
    # --- 1. Data Ingestion ---
    try:
        data = pd.read_csv('final_with_proficiency.csv')
    except FileNotFoundError:
        print("Error: 'final_with_proficiency.csv' not found.")
        return
        
    # --- NEW: Strip whitespace from column names ---
    data.columns = data.columns.str.strip()
        
    print(f"Original data shape: {data.shape}")

    # --- 2. Feature & Target Definition ---
    
    # --- 2a. Static (Non-Time-Series) Features ---
    # These features are constant for each region
    categorical_static_features = ['STATE', 'Metro_zhvi', 'CountyName_zhvi']
    numerical_static_features = [
        'SizeRank_zhvi', 'latest_rent', 'crime_rate_per_100000', 
        'population', 'CountySchoolScore_y', 'Proficiency'
    ]
    static_features = categorical_static_features + numerical_static_features

    # --- 2b. Time-Series (Sequential) Features ---
    # We need to find all the date columns for home value and rent
    all_cols = data.columns.tolist()
    
    # --- REVISED DATE-FINDING LOGIC ---
    # We will use data from 2015 onwards, where we have both ZHVI (price) and ZORI (rent)
    
    # Regex to find 'YYYY-MM-DD_zhvi' and 'YYYY-MM-DD_zori'
    date_regex_zhvi = re.compile(r'^\d{4}-\d{2}-\d{2}_zhvi$')
    date_regex_zori = re.compile(r'^\d{4}-\d{2}-\d{2}_zori$')

    # Find all matching columns and sort them chronologically
    zhvi_cols = sorted([col for col in all_cols if date_regex_zhvi.match(col)])
    zori_cols = sorted([col for col in all_cols if date_regex_zori.match(col)])
    
    # Ensure we have the same number of rent and price columns
    if len(zhvi_cols) != len(zori_cols):
        print(f"Warning: Mismatched time-series columns. Found {len(zhvi_cols)} price cols" +
              f" and {len(zori_cols)} rent cols. Using common subset.")
        
        # Find the common set of dates (e.g., '2015-01-31')
        zhvi_dates = {col.split('_')[0] for col in zhvi_cols}
        zori_dates = {col.split('_')[0] for col in zori_cols}
        common_dates = sorted(list(zhvi_dates.intersection(zori_dates)))
        
        zhvi_cols = [f"{date}_zhvi" for date in common_dates]
        zori_cols = [f"{date}_zori" for date in common_dates]

    if len(zhvi_cols) < 60:
        print(f"Error: Found only {len(zhvi_cols)} time-series columns. Need at least 60 (5 years) for this model.")
        return
    # --- END REVISED LOGIC ---

    # --- 2c. Define Model Time Steps ---
    N_PAST = 48    # Use 48 months (4 years) of history
    N_FUTURE = 12  # Predict the next 12 months (1 year)
    
    # Ensure we have enough data
    if len(zhvi_cols) < N_PAST + N_FUTURE:
        print(f"Error: Not enough date columns ({len(zhvi_cols)}) for {N_PAST} past and {N_FUTURE} future steps.")
        return
        
    # We need 48 + 12 = 60 months of data
    ts_cols_zhvi = zhvi_cols[:N_PAST + N_FUTURE]
    ts_cols_zori = zori_cols[:N_PAST + N_FUTURE]
    
    # --- MODEL UPGRADE: Use 2 Time-Series Features (Price and Rent) ---
    N_FEATURES_TS = 2 
    
    # Define our time-series inputs (X_ts) and targets (y)
    # X_ts: We use *both* price and rent history
    X_ts_data_zhvi = data[ts_cols_zhvi[:N_PAST]]
    X_ts_data_zori = data[ts_cols_zori[:N_PAST]]
    
    # y: We are only forecasting the *price* (zhvi)
    y_data = data[ts_cols_zhvi[N_PAST:N_PAST + N_FUTURE]]

    # --- 3. Preprocessing ---
    
    # --- 3a. Handle Missing Data ---
    # Drop rows where any of our selected columns are NaN
    all_features_to_check = static_features + ts_cols_zhvi + ts_cols_zori
    data.dropna(subset=all_features_to_check, inplace=True)
    
    # --- NEW: Calculate Scores ---
    # This function adds new columns (e.g., 'Safety_Score', 'Overall_Grade')
    # We do this *after* dropping NaNs to ensure we only score valid rows
    data = calculate_scores(data)
    
    # Re-align data after dropping NaNs
    X_ts_data_zhvi = data[ts_cols_zhvi[:N_PAST]]
    X_ts_data_zori = data[ts_cols_zori[:N_PAST]]
    y_data = data[ts_cols_zhvi[N_PAST:N_PAST + N_FUTURE]]
    X_static_data = data[static_features] # <-- FIX: This line was missing
    
    print(f"Data shape after dropping NaNs and scoring: {data.shape}")
    if data.shape[0] == 0:
        print("Error: No data remaining after dropping NaNs.")
        return

    # --- 3b. Preprocessing Static Features ---
    # We need to scale numericals and one-hot encode categoricals
    numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
    categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

    static_preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numerical_static_features),
            ('cat', categorical_transformer, categorical_static_features)
        ])

    # --- 3c. Preprocessing Time-Series Features ---
    # We need to scale the time-series data as well
    ts_scaler_zhvi = StandardScaler()
    ts_scaler_zori = StandardScaler()
    y_scaler = StandardScaler()
    
    # Fit scalers on the full TS dataset (before train/test split)
    # This is a simplification; fitting only on train data is better practice
    X_ts_data_zhvi_scaled = ts_scaler_zhvi.fit_transform(X_ts_data_zhvi)
    X_ts_data_zori_scaled = ts_scaler_zori.fit_transform(X_ts_data_zori)
    
    y_data_scaled = y_scaler.fit_transform(y_data)
    
    # --- STACK TS FEATURES ---
    # Stack the 2 features (price and rent) into one array
    X_ts_data_scaled = np.stack([X_ts_data_zhvi_scaled, X_ts_data_zori_scaled], axis=-1)

    # --- 4. Train/Test Split ---
    # We split all our data on the same index
    indices = np.arange(data.shape[0])
    
    (X_static_train_df, X_static_test_df,
     X_ts_train_scaled, X_ts_test_scaled,
     y_train_scaled, y_test_scaled,
     y_train_unscaled, y_test_unscaled, # <-- FIX: Was missing y_train_unscaled
     indices_train, indices_test) = train_test_split(
        X_static_data,
        X_ts_data_scaled,
        y_data_scaled,
        y_data, # Original unscaled y
        indices,
        test_size=0.2,
        random_state=42
    )

    # --- 5. Apply Static Preprocessing ---
    # Fit the preprocessor on the training data *only*
    X_static_train = static_preprocessor.fit_transform(X_static_train_df)
    # Transform the test data
    X_static_test = static_preprocessor.transform(X_static_test_df)
    
    # --- 6. Reshape TS Data for LSTM ---
    # LSTM needs input as [samples, timesteps, features]
    # Our data is already [samples, timesteps, features] after np.stack
    X_ts_train_reshaped = X_ts_train_scaled
    X_ts_test_reshaped = X_ts_test_scaled
    
    N_STATIC_FEATURES = X_static_train.shape[1]

    # --- 7. Model Definition ---
    model = build_forecasting_model(N_PAST, N_FEATURES_TS, N_STATIC_FEATURES, N_FUTURE)
    model.summary()

    # --- 8. Model Training ---
    print("\nStarting model training...")
    history = model.fit(
        [X_ts_train_reshaped, X_static_train], # List of inputs
        y_train_scaled,
        validation_data=([X_ts_test_reshaped, X_static_test], y_test_scaled),
        epochs=50,
        batch_size=32,
        verbose=1
    )
    print("Training complete.")

    # --- 9. Model Evaluation ---
    test_loss, test_mae = model.evaluate([X_ts_test_reshaped, X_static_test], y_test_scaled, verbose=0)
    print(f"\nModel evaluated on test set.")
    print(f"Test Mean Squared Error (Loss): {test_loss:.2f}")
    print(f"Test Mean Absolute Error (scaled): {test_mae:.2f}")
    
    # --- 10. Example Prediction (Interpretable) ---
    print("\nRunning an example prediction...")
    
    # Get predictions (scaled)
    y_pred_scaled = model.predict([X_ts_test_reshaped, X_static_test])
    
    # Inverse transform to get actual dollar values
    y_pred_unscaled = y_scaler.inverse_transform(y_pred_scaled)
    
    # Compare first prediction in test set
    predicted_forecast_12mo = y_pred_unscaled[0]
    actual_forecast_12mo = y_test_unscaled.iloc[0].values
    
    print("\nExample Region:")
    # Get the original row of data for the first test sample
    example_region_data = data.iloc[indices_test[0]]
    print(example_region_data[['RegionName', 'STATE', 'CountyName_zhvi']])
    
    print("\n--- Risk & Opportunity Scores ---")
    print(f"  Overall Score: {example_region_data['Overall_Score']:.1f} (Grade: {example_region_data['Overall_Grade']})")
    print(f"   Safety Score: {example_region_data['Safety_Score']:.1f} (Grade: {example_region_data['Safety_Grade']})")
    print(f"   School Score: {example_region_data['School_Score']:.1f} (Grade: {example_region_data['School_Grade']})")
    
    print("\n--- Predicted vs. Actual 12-Month Price Forecast ---")
    comparison = pd.DataFrame({
        'Month': range(1, N_FUTURE + 1),
        'Predicted_Price': predicted_forecast_12mo.astype(int),
        'Actual_Price': actual_forecast_12mo.astype(int)
    })
    comparison['Difference'] = comparison['Predicted_Price'] - comparison['Actual_Price']
    
    print(comparison)
    
    # Calculate final, interpretable MAE in dollars
    final_mae = np.mean(np.abs(y_pred_unscaled - y_test_unscaled.values))
    print(f"\nTest Mean Absolute Error (in dollars): ${final_mae:,.2f}")


if __name__ == "__main__":
    main()

Original data shape: (7827, 493)
Calculating Risk/Opportunity Scores...
Scoring complete.
Data shape after dropping NaNs and scoring: (698, 499)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ static_input (InputLayer)     │ (None, 242)               │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ ts_input (InputLayer)         │ (None, 48, 2)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_1 (Dense)               │ (None, 128)               │          31,104 │ static_input[0][0]         │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ lstm (LSTM)                   │ (None, 50)                │          10,600 │ ts_input[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_1 (Dropout)           │ (None, 128)               │               0 │ dense_1[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout (Dropout)             │ (None, 50)                │               0 │ lstm[0][0]                 │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_2 (Dense)               │ (None, 64)                │           8,256 │ dropout_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 20)                │           1,020 │ dropout[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_3 (Dense)               │ (None, 20)                │           1,300 │ dense_2[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 40)                │               0 │ dense[0][0], dense_3[0][0] │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_4 (Dense)               │ (None, 50)                │           2,050 │ concatenate[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ output (Dense)                │ (None, 12)                │             612 │ dense_4[0][0]              │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 54,942 (214.62 KB)

 Trainable params: 54,942 (214.62 KB)

 Non-trainable params: 0 (0.00 B)


Starting model training...
Epoch 1/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 0.9525 - mae: 0.5845 - val_loss: 0.5734 - val_mae: 0.4805
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 3.4446 - mae: 0.5039 - val_loss: 35.2920 - val_mae: 1.0903
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 868.3898 - mae: 2.8387 - val_loss: 0.4514 - val_mae: 0.3434
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.7215 - mae: 0.3699 - val_loss: 0.4643 - val_mae: 0.3335
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.7201 - mae: 0.3373 - val_loss: 0.4450 - val_mae: 0.3172
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.7891 - mae: 0.3480 - val_loss: 0.4194 - val_mae: 0.3007
Epoch 7/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.4399 - mae: 0.2862 - val_loss: 0.3891 - val_mae: 0.2849
Epoch 8/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.5057 - mae: 0.2689 - val_loss: 0.3499 - val_mae: 0.2664
Epoch 9/50
18/18 ━━━━━━━━━━━━━━━━